# 03/15/23 Testing a graycode encoder

In [20]:
import graycode, torch
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
v = graycode.tc_to_gray_code(1)
print(v, bin(v))

1 0b1


In [4]:
mask = 2**torch.arange(15)[None, None, :]
print(mask.size())

torch.Size([1, 1, 15])


In [18]:
def do_graycode(x, bufsize, resolution):
  buf = torch.zeros(bufsize, dtype=torch.int)
  x_adj = torch.tensor(x / resolution).int()
  mask = 2**torch.arange(bufsize).int()
  buf = x_adj.bitwise_and(mask).ne(0).type(torch.int32)
  return buf
  # return x.unsqueeze(-1).bitwise_and(mask).ne(0).byte()
for v in torch.arange(0, 140):
  gray_v = graycode.tc_to_gray_code(v)
  buf = do_graycode(gray_v, 12, resolution = 1)
  print(v)
  print(buf)

tensor(0)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(1)
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(2)
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(3)
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(4)
tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(5)
tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(6)
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(7)
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(8)
tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(9)
tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(10)
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(11)
tensor([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(12)
tensor([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor(13)
tensor([1, 1, 0, 1, 0, 0

/tmp/ipykernel_1902/2791102402.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_adj = torch.tensor(x / resolution).int()


In [32]:
from models.encoders.graycode_encoder import GraycodeEncoder

ge = GraycodeEncoder(128, (32, 32, 64), resolution = 1)

test = torch.arange(20).reshape((4, 5, 1)).repeat(1, 1, 3)
print(f"Test: {test[:,:,0]}")
encoded = ge(test)
print(encoded[0,2])

Test: tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)


In [2]:
from datasets.generic_index_dataset import GenericIndexedModule
from datasets.dataset_utils import pad
from models.identity_module import IdentityModule
from pathlib import Path
import os, pytorch_lightning as pl
import torch

In [5]:
direct = Path("tempdata/SMILES_dataset")
features = ["HSQC", "HYUN_FP"]
feature_handlers = [pad, None]
gim = GenericIndexedModule(direct, features, feature_handlers, len_override = 5)
gim.setup("fit")
model = IdentityModule()
tr_dl = gim.train_dataloader()
val_dl = gim.val_dataloader()

In [8]:
for batch in tr_dl:
  hsqc, _ = batch
  print(hsqc[2])
  break

tensor([[ 1.3680e+02,  5.7058e+00,  6.9806e+03],
        [ 1.2870e+02,  7.1929e+00,  1.5757e+04],
        [ 1.2805e+02,  7.3701e+00,  6.9822e+03],
        [ 1.2706e+02,  7.2030e+00,  5.4609e+03],
        [ 1.1732e+02,  5.2428e+00, -3.4911e+03],
        [ 1.1732e+02,  5.0495e+00, -5.9105e+03],
        [ 1.1433e+02,  6.6775e+00,  6.9806e+03],
        [ 6.4020e+01,  3.3308e+00, -8.7365e+03],
        [ 6.4020e+01,  3.0242e+00, -5.7486e+03],
        [ 5.9850e+01,  3.0122e+00, -6.3924e+03],
        [ 5.9850e+01,  2.9872e+00, -5.0655e+03],
        [ 5.5400e+01,  3.4011e+00, -7.7193e+03],
        [ 5.5400e+01,  3.3711e+00, -5.0655e+03],
        [ 4.4170e+01,  4.3621e+00,  4.7581e+03],
        [ 3.2630e+01,  3.1321e+00, -6.9805e+03],
        [ 3.2630e+01,  2.8483e+00, -8.7769e+03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00